# D3 Challenge

This script is oriented to obtain the results for the D3 Challenge. The main goal is to find the best docked pose for 10 different ligands for an apoprotein called Farnesoid X Receptor (FXR).

## 1. Importing modules

In [1]:
import os
from htmd.ui import *
from subprocess import *
from htmd.molecule.util import maxDistance
from htmd.builder.charmm import defaultParam, defaultTopo
from src.rdock.rdock import *
from src.obabel.babel import *
from src.parameterize.parameterize import *

ffevaluate module is in beta version


/local/fpozoc/miniconda3/envs/dock/lib/python3.6/site-packages/pyemma/util/log.py:65: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  D = yaml.load(f2)



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/

You are on the latest HTMD version (1.15.2).



2019-08-08 15:02:16,955 - htmd.molecule.util - WARNING - Please do not import from htmd.molecule.util. This will be deprecated. Use moleculekit.util instead or moleculekit.dihedral for the dihedral functionality.


In [2]:
#htmd.config(viewer='ngl')

## 2. Defining functions

This first function "dockAllCenters" runs docking of a given protein and ligand for all provided centers.

In [3]:
def dockAllCentersAutoDock(prot, ligand, centers):
    poses = []
    scores = []
    
    for i, center in enumerate(centers):
        currentCenterPoses, currentCenterScoring = dock(prot, ligand, center=center, extent=[ 15., 15., 15. ])
        poses.append(currentCenterPoses)
        scores.append(currentCenterScoring)
        
        print("Finished docking " + str(i+1) + "/" + str(len(centers)))
        
    return poses, scores

In [4]:
def dockAllCentersRdock(protFile, ligandName, centers):
    poses = []
    scores = []
    
    for i, center in enumerate(centers):
        rd = RDock()
        rd.set_radius(8)
        rd.set_protein(protFile) # MOL2 !!
        rd.set_ligand(ligandName) # SD !!
        rd.set_nposes(50)
        rd.set_center(center)
        
        rd.run_docking()
        
        currentCenterPose, currentCenterScoring = rd.get_best_pose()
        
        poses.append(currentCenterPose)
        scores.append(currentCenterScoring)
        
        print(currentCenterScoring)
        print(len(rd.docked_poses))
        print("Finished docking " + str(i+1) + "/" + str(len(centers)))
        
    return poses, scores

This function "buildSystemWithPose" is going to build a system for a given pose obtained from the previous function. It requires a paramFolder to define where the parameters are placed. 

In [5]:
def buildSystemWithPose(prot, pose, systemId, paramFolder, ligandName="MOL"):
    # prepare protein
    prot = proteinPrepare(prot)
    prot.set('segid', 'P', sel='protein')
    prot.set('segid', 'W', sel='water')
    
    # prepare ligand (pose)
    pose.set('segid','L')
    pose.set('resname', ligandName) 
    
    mol = Molecule(name='combo')
    mol.append(prot)
    mol.append(pose)
    mol.center()
    
    # distance + extra
    D = maxDistance(mol, 'all') + 10
    
    # solvate
    smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])
    
    # build system    
    param = defaultParam()
    #It takes the name of the files by default, so do not change them
    param.append(paramFolder + "mol.prm")

    topo = defaultTopo()
    topo.append(paramFolder + "mol.rtf")
    
    bmol = charmm.build(smol, param=param, topo=topo, outdir="build-{}".format(systemId))
    
    return bmol

## 3. Docking

Loading protein in pdb format:

In [6]:
prot = Molecule('project_data/FXR_APO_structure_D3R_GC2.pdb')
prot.view()

/local/fpozoc/miniconda3/envs/dock/lib/python3.6/site-packages/pyemma/__init__.py:91: UserWarning: You are not using the latest release of PyEMMA. Latest is 2.5.6, you have 2.4.
  .format(latest=latest, current=current), category=UserWarning)
2019-08-08 15:02:17,175 - moleculekit.molecule - WARNING - Alternative atom locations detected. Only altloc A was kept. If you prefer to keep all use the keepaltloc="all" option when reading the file.
2019-08-08 15:02:17,178 - moleculekit.molecule - INFO - Removed 15 atoms. 2054 atoms remaining in the molecule.


Load all ligands in mol2 format:

In [7]:
ligands = []
ligandCount = 10
for i in range(ligandCount):
    fname = "project_data/ligand_FXR_{}.sdf.mol2".format(i+1)
    ligands.append([Molecule(fname), fname])
    
ligands[0][0].view()

Defining binding pocket centers extracted with DeepSite (in playmolecule):

In [8]:
centers = [[43.9, 9.9, 3.2],
           [29.9, 7.9, 7.2],
           [25.9, 3.9, -2.8],
           [21.9, 3.9, -12.8],
           [29.9, 19.9, -0.8]]

Now, we will define the output that we would like to obtain.

In [9]:
saveAllProtLigandToFile = True #True to save all the poses and proteins in a pdb file.
saveBestPosesToFile = True #True to save only the best pose in a pdb file.

# write csv header if saving to file
scoresFile = None
if saveBestPosesToFile: 
    scoresFile = open('ligand_scores.csv','w+')
    scoresFile.write("kcal1,rmsd_lb1,rmsd_ub1,rdock_score\n")

Once we defined all the previous variables now we are going to run the docking.

In [10]:
if not os.path.exists("allposes/autodock"):
    os.makedirs("allposes/autodock")
if not os.path.exists("allposes/rdock"):
    os.makedirs("allposes/rdock")

In [ ]:
for l, ligand in enumerate(ligands): 
    #Defining variables
    bestKcalPoseAutoDock, bestPoseRdock = None, None
    bestKcalScoresAutoDock, bestScoreRdock = [float("inf")]*3, float("inf")
    
 
    #Here we will run a docking for every ligand and every
    
    #Docking with Autodock VINA
    poses, scores = dockAllCentersAutoDock(prot, ligand[0], centers)
    for i, poseListPerCenter in enumerate(poses):
        for j, pose in enumerate(poseListPerCenter):
            if saveAllProtLigandToFile:
                mol = Molecule()
                mol.append(prot)
                mol.append(pose)
                mol.write("allposes/autodock/combo_ligand_{}_center_{}_pose_{}_autodock.pdb".format(l+1, i+1, j+1))
                
            if scores[i][j][0] < bestKcalScoresAutoDock[0]:
                bestKcalScoresAutoDock = scores[i][j]
                bestKcalPoseAutoDock = pose
                
    
    #If we select this option we will save the best pose in a file in mol2 format.
    if saveBestPosesToFile:
        # write pdb files
        bestKcalPoseAutoDock.write("selected_autodock_{}.mol2".format(l+1))
        
        # and csv with scores
        # format:
        # kcal1, rmsd_lb1, rmsd_ub1, kcal2, rmsd_lb2, rmsd_ub2
        # where 1 is the pose selected by best kcal and 2 is the pose selected by best rmsd        
        allScores = []
        allScores.extend(bestKcalScoresAutoDock)

        scoresFile.write(','.join(map(str, allScores))+"\n")    
        
    #bmol = buildSystemWithPose(prot, bestKcalPose, "project_data/param-ligand_FXR_1.sdf/parameters/CGenFF_2b6/b3lyp-cc-pVDZ-water/")
    #bmol.view(viewer='vmd')
    
    #tmpMol = bmol
    #tmpBestPose = bestKcalPose
    
if saveBestPosesToFile:
    scoresFile.close()

2019-08-08 15:02:59,325 - htmd.dock - INFO - Charges detected in ligand and will be used for docking.


Finished docking 1/5


2019-08-08 15:04:00,558 - htmd.dock - INFO - Charges detected in ligand and will be used for docking.


Finished docking 2/5


2019-08-08 15:05:17,863 - htmd.dock - INFO - Charges detected in ligand and will be used for docking.


Finished docking 3/5


2019-08-08 15:06:53,697 - htmd.dock - INFO - Charges detected in ligand and will be used for docking.


Finished docking 4/5


2019-08-08 15:08:32,200 - htmd.dock - INFO - Charges detected in ligand and will be used for docking.


In [ ]:
for l, ligand in enumerate(ligands): 
    #Defining variables
    bestKcalPoseAutoDock, bestPoseRdock = None, None
    bestKcalScoresAutoDock, bestScoreRdock = [float("inf")]*3, float("inf")
    
 
    #Here we will run a docking for every ligand and every
    
    #Docking with Autodock VINA
    poses, scores = dockAllCentersAutoDock(prot, ligand[0], centers)
    for i, poseListPerCenter in enumerate(poses):
        for j, pose in enumerate(poseListPerCenter):
            if saveAllProtLigandToFile:
                mol = Molecule()
                mol.append(prot)
                mol.append(pose)
                mol.write("allposes/autodock/combo_ligand_{}_center_{}_pose_{}_autodock.pdb".format(l+1, i+1, j+1))
                
            if scores[i][j][0] < bestKcalScoresAutoDock[0]:
                bestKcalScoresAutoDock = scores[i][j]
                bestKcalPoseAutoDock = pose
                
    #After that we are going to do a docking with RDOCK. RDOCK needs proteins in .mol2 and ligands in .sd
    #For this reason, we will convert the files using babel(called from python)
    
    bb = Babel()
    bb.set_input_file(ligand[1])
    
    output_sd = '.'.join(ligand[1].split(".")[:-1]) + ".sd"
    bb.set_output_file(output_sd)
    
    bb.protonate(True)
    bb.run()
    
    #Docking with RDOCK
    poses, scores = dockAllCentersRdock('project_data/FXR_APO_structure_D3R_GC2.mol2', output_sd, centers)
    for i, pose in enumerate(poses):
        if saveAllProtLigandToFile:
            mol = Molecule()
            mol.append(prot)
            mol.append(pose)
            mol.write("allposes/rdock/combo_ligand_{}_center_{}_pose_1_rdock.pdb".format(l+1, i+1))
        
        if scores[i] < bestScoreRdock:
            bestScoreRdock = scores[i]
            bestPoseRdock = pose
    
    #If we select this option we will save the best pose in a file in mol2 format.
    if saveBestPosesToFile:
        # write pdb files
        bestKcalPoseAutoDock.write("selected_autodock_{}.mol2".format(l+1))
        bestPoseRdock.write("selected_rdock_{}.mol2".format(l+1))
        
        # and csv with scores
        # format:
        # kcal1, rmsd_lb1, rmsd_ub1, kcal2, rmsd_lb2, rmsd_ub2
        # where 1 is the pose selected by best kcal and 2 is the pose selected by best rmsd        
        allScores = []
        allScores.extend(bestKcalScoresAutoDock)
        allScores.extend([bestScoreRdock])

        scoresFile.write(','.join(map(str, allScores))+"\n")    
        
    #bmol = buildSystemWithPose(prot, bestKcalPose, "project_data/param-ligand_FXR_1.sdf/parameters/CGenFF_2b6/b3lyp-cc-pVDZ-water/")
    #bmol.view(viewer='vmd')
    
    #tmpMol = bmol
    #tmpBestPose = bestKcalPose
    
if saveBestPosesToFile:
    scoresFile.close()

## 4. Building protein-ligand system

Once we have the results of the docking we are going to build a system protein-ligand with the best poses obtained from the previous algorithms. 

In [ ]:
#creation of parameters for the build process calling parameterize commands
builds = []
for i in range(ligandCount):
    name_ad = "selected_autodock_{}.mol2".format(i+1)
    name_rd = "selected_rdock_{}.mol2".format(i+1)
    
    # protonate rdock pose
    bb = Babel()
    bb.set_input_file(name_rd)
    bb.set_output_file(name_rd)
    
    bb.protonate(True)
    bb.run()
    
    # parameterize autodock pose
    pr = Parameterize()
    pr.set_input(name_ad)
    pr.set_output("./params-ad-{}".format(i+1))
    pr.no_esp()
    pr.no_torsions()
    pr.no_min()
    pr.run()
    
    # parameterize rdock pose
    pr.set_input(name_rd)
    pr.set_output("./params-rd-{}".format(i+1))
    pr.run()
    
    folderAd = "params-ad-{}/parameters/CGenFF_2b6/b3lyp-cc-pVDZ-water/".format(i+1)
    folderRd = "params-rd-{}/parameters/CGenFF_2b6/b3lyp-cc-pVDZ-water/".format(i+1)
    
    lad = Molecule(folderAd + "mol.mol2")
    lrd = Molecule(folderRd + "mol.mol2")
    
    # running the Charmm building for the best poses using the function buildSystemWithPose    
    builds.append(buildSystemWithPose(prot, lad, "ad-{}".format(i+1), folderAd))
    builds.append(buildSystemWithPose(prot, lrd, "rd-{}".format(i+1), folderRd))

## 5. Simulation

After generating all the files needed the next step will be run a simulation for the poses produced on the previous steps in order to analyse their stability.

The first step is running an equilibration of 40 ns with a temperature of 300.

In [ ]:
from htmd.protocols.equilibration_v2 import Equilibration

#setting parameters for the equilibration
md = Equilibration()
md.runtime = 40
md.timeunits = 'ns'
md.temperature = 300
md.useconstantratio = False

local = LocalGPUQueue()

#run the equilibration for each build folder created before
for i, b in enumerate(builds):
    md.write('./build-{}/'.format(i+1), './equil/{}/'.format(i+1))
    local.submit('./equil/{}/'.format(i+1))
                 
local.wait()

Then, we should run a production of 60 ns with a temperature of 300. 

In [ ]:
from htmd.protocols.production_v4 import Production
#setting parameters for the production
md = Production()
md.runtime = 60
md.timeunits = 'ns'
md.temperature = 300
md.acemd.bincoordinates = 'output.coor'
md.acemd.extendedsystem = 'output.xsc'

local = LocalGPUQueue()

#run a production for each equilibration folder created before
for i, b in enumerate(builds):
    md.write('./equil/{}/'.format(i+1), './prod/{}/'.format(i+1))
    local.submit('./prod/{}/'.format(i+1))
                 
local.wait()

Following all the previous process we will have one single trajectory for each best-pose obtained for each ligand (and also for each algorithm tested) in the docking. In case we wanted to analyse the stability of the ligand it will be interesting to repeat these simulations in order to obtain more trajectories to be able to have enough data. 